# Multi-layer merge script

This script runs a single layer join on hex grid. Uses Kate's generated hexes and Inputs.<br>
There is no cleaning and type controll. No optimization/vectorization/parallelization. 

In [1]:
# Import modules
import geopandas as gpd
import fiona

Modify below cell to indicate path to database, hex level and joined layer

In [2]:
# Define path to hex grid database
hex_gdb = r'C://Research/Grid_effort/H3_5_10_Grd.gdb'
# Define hex level
hexLevel = '5'

# Define the path to the geodatabase with joined layer
inputs_gdb = r'C://Research/Grid_effort/H3Grid_Inputs.gdb'
# Names of layers to join in the list, determines the order of joins
layers_to_join = ['tj_2021_us_st_cnt', 'Estuarine_Drainage_Areas', 'WBDHU8', 'dtl_cnty_Census_ESRI', 'WBDHU12']

Run below cell to perform merge and save result

In [4]:
# Read in the hex polygon grid layer as the base layer
base_layer = gpd.read_file(hex_gdb, layer='H3_'+hexLevel)
base_layer.drop(columns=['Shape_Length','Shape_Area'],inplace=True)
# Read the joined layers into dictionary
layer_gdfs = {layer: gpd.read_file(inputs_gdb, layer=layer) for layer in layers_to_join}
# Check if garbage data is present and try to remove it
for name, gdf in layer_gdfs.items():
    # Ensure the CRS is the same between the base layer and the current layer
    if not gdf.crs == base_layer.crs:
        gdf = gdf.to_crs(base_layer.crs)
    # Check if garbage data is present and remove it
    if 'Shape_Length' in gdf.columns:
        gdf.drop('Shape_Length', axis=1, inplace=True)
    if 'Shape_Area' in gdf.columns:
        gdf.drop('Shape_Area', axis=1, inplace=True)
    # Perform spatial join
    # here a modification migh be needed, to include different predicates parameter, to differ results
    # more here: https://shapely.readthedocs.io/en/latest/manual.html#binary-predicates
    base_layer = gpd.sjoin(base_layer, gdf, how="left")
    # Drop the `index_right` column if it exists
    if 'index_right' in base_layer.columns:
        base_layer.drop('index_right', axis=1, inplace=True)
# Save the result in work path
base_layer.to_file('h'+hexLevel+'_multipleMerges.gdb',driver='OpenFileGDB')